In [6]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt 

Для каждого дня в таблице orders рассчитайте следующие показатели:
Выручку, полученную в этот день.
Суммарную выручку на текущий день.
Прирост выручки, полученной в этот день, относительно значения выручки за предыдущий день.

In [ ]:
WITH
list_product AS (
    SELECT
        creation_time::DATE AS date,
        order_id,
        unnest(product_ids) AS product_id
    FROM
        orders
    WHERE 
        order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
),
price_order AS (
    SELECT
        LP.date AS date,
        LP.order_id AS order_id,
        SUM(P.price) AS revenue_order
    FROM
        list_product AS LP
        LEFT JOIN products AS P
        ON LP.product_id = P.product_id
    GROUP BY
        order_id, date
),
t1 AS (
    SELECT
        date,   
        SUM(revenue_order) AS revenue
    FROM
        price_order
    GROUP BY
        date
)
SELECT
    date,
    revenue,
    SUM(revenue) OVER(order by date) total_revenue,
    ROUND((100*revenue::decimal / LAG(revenue, 1) OVER(order by date)-100), 2) AS revenue_change
FROM
    t1
ORDER BY
    date

In [2]:
file_path1 = ('data/22.csv')
df1 = pd.read_csv(file_path1)
print(df1)

        date  revenue  total_revenue  revenue_change
0   24/08/22    49924          49924             NaN
1   25/08/22   430860         480784          763.03
2   26/08/22   534766        1015550           24.12
3   27/08/22   817053        1832603           52.79
4   28/08/22  1133370        2965973           38.71
5   29/08/22  1279891        4245864           12.93
6   30/08/22  1279377        5525241           -0.04
7   31/08/22  1312720        6837961            2.61
8   01/09/22  1406101        8244062            7.11
9   02/09/22  1907107       10151169           35.63
10  03/09/22  2210988       12362157           15.93
11  04/09/22  2294009       14656166            3.75
12  05/09/22  1784690       16440856          -22.20
13  06/09/22  1330931       17771787          -25.43
14  07/09/22  1807800       19579587           35.83
15  08/09/22  2099508       21679095           16.14


Динамика общей выручки.
![Image](./charts/14.png)

Динамика ежедневной выручки. 
![Image](./charts/15.png)

В промежутке между 30 августом и 1 сентября наблюдается заметное снижение ежедневной выручки.

Для каждого дня в таблицах orders и user_actions рассчитайте следующие показатели:
Выручку на пользователя (ARPU) за текущий день.
Выручку на платящего пользователя (ARPPU) за текущий день.
Выручку с заказа, или средний чек (AOV) за текущий день.

In [ ]:
WITH
list_product AS (
    SELECT
        creation_time::DATE AS date,
        order_id,
        unnest(product_ids) AS product_id
    FROM
        orders
    WHERE 
        order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
),
price_order AS (
    SELECT
        LP.date AS date,
        LP.order_id AS order_id,
        SUM(P.price) AS revenue_order
    FROM
        list_product AS LP
        LEFT JOIN products AS P
        ON LP.product_id = P.product_id
    GROUP BY
        order_id, date
),
count_user AS (
    SELECT
        COUNT(DISTINCT(user_id)) AS user_count,
        time::date AS date
    FROM
        user_actions
    GROUP BY
        date
),
paying_user AS (
    SELECT
        time::date AS date,
        COUNT(DISTINCT(user_id)) AS  paying_user
    FROM
        user_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
    GROUP BY
        date
),
count_order AS (
    SELECT
        creation_time::date AS date,
        COUNT(order_id) AS order_counts
    FROM
        orders
    WHERE
        order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
    GROUP BY
        date
),
t1 AS (
    SELECT
        date,   
        SUM(revenue_order) AS revenue
    FROM
        price_order
    GROUP BY
        date
)
SELECT
    t1.date AS date,
    ROUND(t1.revenue::decimal / CU1.user_count, 2) AS ARPU,
    ROUND(t1.revenue::decimal / PU.paying_user, 2) AS ARPPU,
    ROUND(t1.revenue::decimal / CO.order_counts, 2) AS AOV
FROM
    t1 AS t1
    LEFT JOIN count_user AS CU1
    ON t1.date = CU1.date
    LEFT JOIN paying_user AS PU
    ON t1.date = PU.date
    LEFT JOIN count_order AS CO
    ON t1.date = CO.date
ORDER BY
    t1.date

In [3]:
file_path2 = ('data/1.csv')
df2 = pd.read_csv(file_path2)
print(df2)

        date    arpu   arppu     aov
0   24/08/22  372.57  393.10  361.77
1   25/08/22  508.09  525.44  406.86
2   26/08/22  452.04  470.33  369.57
3   27/08/22  509.38  527.81  381.62
4   28/08/22  528.38  544.10  378.04
5   29/08/22  559.15  581.24  391.76
6   30/08/22  546.74  567.85  379.52
7   31/08/22  517.63  540.21  384.96
8   01/09/22  499.33  518.86  381.26
9   02/09/22  537.67  556.17  381.35
10  03/09/22  565.90  582.76  387.28
11  04/09/22  541.55  558.97  381.70
12  05/09/22  512.84  530.84  381.75
13  06/09/22  475.33  492.75  385.67
14  07/09/22  496.65  514.02  378.44
15  08/09/22  521.10  536.68  383.54


Динамика ежедневного изменения показателей. 
![Image](./charts/16.png)

Наибольший разброс значений имеют метрики ARPU и ARPPU. Значение количества платящих пользователей и всех пользователей сервиса примерно одинаковое. 

По таблицам orders и user_actions для каждого дня рассчитайте следующие показатели:
Накопленную выручку на пользователя (Running ARPU).
Накопленную выручку на платящего пользователя (Running ARPPU).
Накопленную выручку с заказа, или средний чек (Running AOV).

In [ ]:
WITH
-- Список продуктов в заказах
list_product AS (
    SELECT
        creation_time::DATE AS date,
        order_id,
        unnest(product_ids) AS product_id
    FROM
        orders
    WHERE 
        order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
),
-- Выручка по каждому заказу
price_order AS (
    SELECT
        LP.date AS date,
        LP.order_id AS order_id,
        SUM(P.price) AS revenue_order
    FROM
        list_product AS LP
        LEFT JOIN products AS P
        ON LP.product_id = P.product_id
    GROUP BY
        order_id, date
),
-- Дневная выручка
daily_revenue AS (
    SELECT
        date,   
        SUM(revenue_order) AS revenue
    FROM
        price_order
    GROUP BY
        date
), 
-- Накопительная выручка
cumulative_revenue AS (
    SELECT
        date,
        SUM(revenue) OVER (order by date) AS total_revenue
    FROM 
        daily_revenue
),
-- Даты первого действия пользователей
user_start_dates AS (
    SELECT
        user_id,
        MIN(time)::date AS start_date
    FROM
        user_actions
    GROUP BY
        user_id
),
-- Ежедневное количество новых пользователей
daily_new_users AS (
    SELECT
        start_date AS date,
        COUNT(user_id) AS new_users
    FROM
        user_start_dates
    GROUP BY
        start_date
),
-- Накопленное количество пользователей
count_user AS (
    SELECT
        date,
        SUM(new_users) OVER(order by date) AS user_count
    FROM
        daily_new_users
),
-- Даты первых покупок пользователей
first_purchase_dates AS (
    SELECT 
        user_id,
        MIN(time::date) AS date
    FROM 
        user_actions
    WHERE 
        order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
    GROUP BY 
        user_id
),
-- Ежедневное количество новых платящих пользователей
daily_new_paying_users AS (
    SELECT
        date,
        COUNT(user_id) AS new_paying_users
    FROM
        first_purchase_dates
    GROUP BY
        date
),
-- Накопленное количество платящих пользователей
paying_users_cumulative AS (
    SELECT
        date,
        SUM(new_paying_users) OVER (ORDER BY date) AS pay_u
    FROM
        daily_new_paying_users
),
-- Ежедневное количество заказов (без отмененных)
count_order AS (
    SELECT
        creation_time::date AS date,
        COUNT(order_id) AS order_counts
    FROM
        orders
    WHERE
        order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
    GROUP BY
        date
),
-- Накопленное количество заказов
total_orders AS (
    SELECT
        date,
        SUM(order_counts) OVER (ORDER BY date) AS total_c
    FROM
        count_order
)
SELECT
    cr.date AS date,
    ROUND(cr.total_revenue::DECIMAL / cu.user_count, 2) AS running_arpu,
    ROUND(cr.total_revenue::DECIMAL / puc.pay_u, 2) AS running_arppu,
    ROUND(cr.total_revenue::DECIMAL / to1.total_c, 2) AS running_aov
FROM
    cumulative_revenue AS cr
    LEFT JOIN count_user AS cu
    ON cr.date = cu.date
    LEFT JOIN paying_users_cumulative AS puc 
    ON cr.date = puc.date
    LEFT JOIN total_orders AS to1 
    ON cr.date = to1.date
ORDER BY 
    date

In [4]:
file_path3 = ('data/2.csv')
df3 = pd.read_csv(file_path3)
print(df3)

        date  running_arpu  running_arppu  running_aov
0   24/08/22        372.57         393.10       361.77
1   25/08/22        499.26         517.53       401.66
2   26/08/22        512.90         530.87       384.10
3   27/08/22        571.80         590.21       382.99
4   28/08/22        632.13         649.72       381.08
5   29/08/22        707.53         726.29       384.24
6   30/08/22        766.86         786.40       383.14
7   31/08/22        792.81         813.46       383.49
8   01/09/22        813.18         832.90       383.11
9   02/09/22        844.17         863.05       382.77
10  03/09/22        886.24         904.39       383.57
11  04/09/22        921.71         938.78       383.28
12  05/09/22        950.45         967.17       383.11
13  06/09/22        970.18         986.72       383.30
14  07/09/22        992.38        1007.85       382.85
15  08/09/22       1012.99        1028.03       382.91


Динамика ежедневного изменения показателей. 
![Image](./charts/17.png)

Значения ARPU и ARPPU увеличиваются, AOV остается без изменений. Можно также предположить, что со временем растёт число заказов на одного пользователя. 

In [ ]:
WITH
product_list AS (
    SELECT
        order_id,
        creation_time::date AS date,
        UNNEST(product_ids) AS product_id
    FROM
        orders
    WHERE 
        creation_time >= '2022-08-26' 
        AND creation_time < '2022-09-09' 
), 
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
        AND time >= '2022-08-26' 
        AND time < '2022-09-09' 
),
daiy_active_users AS (
    SELECT
        COUNT(DISTINCT(user_id)) AS user_count,
        DATE_PART('isodow', time) AS weekday_number,
        TO_CHAR(time, 'Day') AS weekday
    FROM
        user_actions
    WHERE 
        time >= '2022-08-26' 
        AND time < '2022-09-09' 
    GROUP BY
        weekday, 
        weekday_number
),
daily_paying_users AS (
    SELECT
        COUNT(DISTINCT(user_id)) AS paying_user_count,
        TO_CHAR(time, 'Day') AS weekday,
        DATE_PART('isodow', time) AS weekday_number
    FROM
        user_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
        AND time >= '2022-08-26' 
        AND time < '2022-09-09'
    GROUP BY
        weekday,
        weekday_number
),
orders_count AS (
    SELECT
        COUNT(order_id) AS order_count,
        DATE_PART('isodow', creation_time) AS weekday_number,
        TO_CHAR(creation_time, 'Day') AS weekday
    FROM
        orders   
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
        AND creation_time >= '2022-08-26' 
        AND creation_time < '2022-09-09'
    GROUP BY
        weekday, 
        weekday_number
),
daiy_revenue AS (
    SELECT
        DATE_PART('isodow', t1.date) AS weekday_number,
        TO_CHAR(t1.date, 'Day') AS weekday,
        SUM(t2.price) AS revenue
    FROM
        product_list AS t1
        LEFT JOIN products AS t2
        ON t1.product_id = t2.product_id
    WHERE
        t1.order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        weekday, 
        weekday_number
)
SELECT
    t3.weekday,
    t3.weekday_number,
    ROUND(t3.revenue::decimal / t4.user_count, 2) AS arpu,
    ROUND(t3.revenue::decimal / t5.paying_user_count, 2) AS arppu,
    ROUND(t3.revenue::decimal / t6.order_count, 2) AS aov
    
FROM
    daiy_revenue AS t3 
    LEFT JOIN daiy_active_users AS t4
    ON t3.weekday_number = t4.weekday_number
    LEFT JOIN daily_paying_users AS t5
    ON t3.weekday_number = t5.weekday_number
    LEFT JOIN orders_count AS t6
    ON t3.weekday_number = t6.weekday_number
ORDER BY
    weekday_number

In [3]:
file_path4 = ('data/3.csv')
df4 = pd.read_csv(file_path4)
print(df4)

     weekday  weekday_number    arpu   arppu     aov
0  Monday                1.0  555.98  575.18  385.87
1  Tuesday               2.0  528.94  548.04  382.63
2  Wednesday             3.0  528.90  548.33  381.16
3  Thursday              4.0  533.98  551.37  382.62
4  Friday                5.0  534.79  553.21  378.70
5  Saturday              6.0  578.53  595.48  385.74
6  Sunday                7.0  566.23  583.38  380.48


Динамика еженедельного изменения показателей. 
![Image](./charts/19.png)

Пик метрик ARPU и ARPPU прихоидится на субботу. Часто в выходные дни в сервисах доставки показатели имеют более высокие результаты. Метрика AOV почти не именяется, это может быть связанно с тем, что средний чек примерно одинаковый. 

Для каждого дня в таблицах orders и user_actions рассчитайте следующие показатели:
Выручку, полученную в этот день.
Выручку с заказов новых пользователей, полученную в этот день.
Долю выручки с заказов новых пользователей в общей выручке, полученной за этот день.
Долю выручки с заказов остальных пользователей в общей выручке, полученной за этот день.

In [ ]:
WITH
product_list AS (
    SELECT
        order_id,
        creation_time::date AS date,
        UNNEST(product_ids) AS product_id
    FROM
        orders
), 
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
), 
user_start_dates AS (
    SELECT
        user_id,
        MIN(time)::date AS start_date
    FROM
        user_actions
    GROUP BY
        user_id
),
daily_new_users AS (
    SELECT
        a1.user_id, 
        a1.order_id,
        a1.time::date AS date
    FROM
        user_actions AS a1
        JOIN user_start_dates AS a2
        ON a1.user_id = a2.user_id AND a1.time::date = a2.start_date
    WHERE 
        order_id NOT IN (SELECT order_id FROM canceled_orders)
),
new_users_revenue AS (
    SELECT
        t1.date AS date,
        SUM(t3.price) AS new_users_revenue
    FROM
        product_list AS t1
        JOIN daily_new_users t2 
        ON t1.order_id = t2.order_id
        LEFT JOIN products AS t3
        ON t1.product_id = t3.product_id
    WHERE
        t1.order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        t1.date
), 
revenue AS (
    SELECT
        t1.date AS date,
        SUM(t3.price) AS users_revenue
    FROM
        product_list AS t1
        LEFT JOIN products AS t3
        ON t1.product_id = t3.product_id
    WHERE
        t1.order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        t1.date
)
SELECT
    b2.date,
    b2.users_revenue AS revenue,
    b1.new_users_revenue,
    ROUND(100*b1.new_users_revenue::decimal / b2.users_revenue, 2) AS new_users_revenue_share,
    100 - ROUND(100*b1.new_users_revenue::decimal / b2.users_revenue, 2) AS old_users_revenue_share
FROM
    revenue AS b2
    JOIN new_users_revenue AS b1
    ON b2.date = b1.date
ORDER BY 
    b2.date

In [4]:
file_path5 = ('data/4.csv')
df5 = pd.read_csv(file_path5)
print(df5)

        date  revenue  new_users_revenue  new_users_revenue_share  \
0   24/08/22    49924              49924                   100.00   
1   25/08/22   430860             417333                    96.86   
2   26/08/22   534766             463326                    86.64   
3   27/08/22   817053             619318                    75.80   
4   28/08/22  1133370             801162                    70.69   
5   29/08/22  1279891             717374                    56.05   
6   30/08/22  1279377             656429                    51.31   
7   31/08/22  1312720             720381                    54.88   
8   01/09/22  1406101             757287                    53.86   
9   02/09/22  1907107            1017824                    53.37   
10  03/09/22  2210988            1079256                    48.81   
11  04/09/22  2294009            1063997                    46.38   
12  05/09/22  1784690             714459                    40.03   
13  06/09/22  1330931             

Динамика ежедневного изменения показателей. 
![Image](./charts/20.png)

Спустя две недели после запуска сервиса показатель выручки от новых пользователей остается на довольно высоком уровне.

Для каждого товара, представленного в таблице products, за весь период времени в таблице orders рассчитайте следующие показатели:
Суммарную выручку, полученную от продажи этого товара за весь период.
Долю выручки от продажи этого товара в общей выручке, полученной за весь период.

In [ ]:
WITH
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
product_list AS (
    SELECT
        creation_time::date AS date,
        order_id,
        UNNEST(product_ids) AS product_id
    FROM
        orders
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
),
revenue_product AS (
    SELECT
        t2.name AS product_name,
        SUM(t2.price) AS revenue
    FROM
        product_list AS t1
        LEFT JOIN products AS t2
        ON t1.product_id = t2.product_id
    GROUP BY
        t2.name
),
total_revenue AS (
    SELECT
        SUM(revenue) AS total
    FROM 
        revenue_product
),
A AS (
    SELECT
        product_name AS product_category,
        revenue AS revenue1,
        ROUND(100 * revenue / (SELECT total FROM total_revenue), 2) AS share_in_revenue_1
    FROM
        revenue_product
),
B AS (
    SELECT
        CASE
            WHEN share_in_revenue_1 < 0.5 THEN 'ДРУГОЕ' 
            ELSE product_category
        END AS product_name,
        SUM(revenue1) AS revenue,
        SUM(share_in_revenue_1) AS share_in_revenue
    FROM A
    GROUP BY
        product_name
)
SELECT
    product_name,
    revenue,
    share_in_revenue
FROM 
    B
ORDER BY 
    revenue DESC

In [5]:
file_path6 = ('data/5.csv')
df6 = pd.read_csv(file_path6)
print(df6)

                product_name  revenue  share_in_revenue
0                    свинина  1353600              6.24
1                     ДРУГОЕ  1225387              5.64
2                     курица  1171140              5.40
3            масло оливковое  1163250              5.37
4                   говядина   977170              4.51
..                       ...      ...               ...
63                       рис   118930              0.55
64  чай травяной в пакетиках   112580              0.52
65               чайный гриб   112448              0.52
66           масло кунжутное   109750              0.51
67       жевательная резинка   109650              0.51

[68 rows x 3 columns]


Динамика изменения показателей. 
![Image](./charts/21.png)

Товары с наибольшей долей выручки это свинина, курица, а также товары из категории  ДРУГОЕ. 

Для каждого дня в таблицах orders и courier_actions рассчитайте следующие показатели:
Выручку, полученную в этот день.
Затраты, образовавшиеся в этот день.
Сумму НДС с продажи товаров в этот день.
Валовую прибыль в этот день (выручка за вычетом затрат и НДС).
Суммарную выручку на текущий день.
Суммарные затраты на текущий день.
Суммарный НДС на текущий день.
Суммарную валовую прибыль на текущий день.
Долю валовой прибыли в выручке за этот день (долю п.4 в п.1).
Долю суммарной валовой прибыли в суммарной выручке на текущий день (долю п.8 в п.5).

In [ ]:
WITH
-- Все отмененные заказы
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
-- Выбираются все успешно доставленные заказы
courier_daily_stats AS (
    SELECT
        time::date AS date,
        courier_id,
        COUNT(order_id) FILTER (WHERE action = 'deliver_order') AS deliver_orders,
        COUNT(order_id) FILTER (WHERE action = 'accept_order') AS accepted_orders
    FROM
        courier_actions
    WHERE 
        order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        date, 
        courier_id
), 
courier_stats_aggregated AS (
    SELECT
        date,
        SUM(deliver_orders) AS total_deliver_orders,
        SUM(accepted_orders) AS total_accepted_orders
    FROM
        courier_daily_stats
    GROUP BY
        date
),
-- Список продуктов в заказах
product_list AS (
    SELECT
        creation_time::date AS date,
        order_id,
        UNNEST(product_ids) AS product_id
    FROM
        orders
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
),
-- Выручка по продуктам
revenue_product AS (
    SELECT
        t1.date AS date,
        t1.order_id AS order_id,
        t2.name AS product_name,
        t2.price AS price
    FROM
        product_list AS t1
        LEFT JOIN products AS t2
        ON t1.product_id = t2.product_id
),
revenue_day AS (
    SELECT
        date,
        SUM(price) AS revenue
    FROM
        revenue_product
    GROUP BY
        date
    ORDER BY
        date
),
-- Ежедневные бонусы
daily_bonus AS (
    SELECT
        date,
        SUM(
            CASE 
            WHEN deliver_orders >= 5 THEN 
                CASE 
                    WHEN DATE_PART('year', date) = 2022 AND DATE_PART('month', date) = 8 THEN 400
                    WHEN DATE_PART('year', date) = 2022 AND DATE_PART('month', date) = 9 THEN 500
                    ELSE 0
                END
            ELSE 0
        END) AS bonus
    FROM
        courier_daily_stats
    GROUP BY
        date
), 
-- Расчет затрат
expenses AS (
        SELECT
            t1.date AS date,
            CASE
                WHEN DATE_PART('year', t1.date) = 2022 AND DATE_PART('month', t1.date) = 8 THEN t2.bonus + 120000 + t1.total_accepted_orders * 140 + t1.total_deliver_orders * 150
                WHEN DATE_PART('year', t1.date) = 2022 AND DATE_PART('month', t1.date) = 9 THEN t2.bonus + 150000 + t1.total_accepted_orders  * 115 + t1.total_deliver_orders * 150
            ELSE 0
            END AS expense
        FROM
            courier_stats_aggregated  AS t1
            LEFT JOIN daily_bonus  AS t2
            ON t1.date = t2.date
), 
-- Расчет НДС по продуктам
product_taxes AS ( 
SELECT
    date,
    product_name,
    price,
    CASE
        WHEN product_name IN ('сахар', 'сухарики', 'сушки', 'семечки','масло льняное', 'виноград', 'масло оливковое', 'арбуз', 'батон', 'йогурт', 'сливки', 'гречка', 'овсянка', 'макароны', 'баранина', 'апельсины', 'бублики', 'хлеб', 'горох', 'сметана', 'рыба копченая', 'мука', 'шпроты', 'сосиски', 'свинина', 'рис', 'масло кунжутное', 'сгущенка', 'ананас', 'говядина', 'соль', 'рыба вяленая', 'масло подсолнечное', 'яблоки', 'груши', 'лепешка', 'молоко', 'курица', 'лаваш', 'вафли', 'мандарины') 
        THEN ROUND((price * 10) / 110, 2)
        ELSE ROUND((price * 20) / 120, 2)
    END AS revenue_tax
    FROM
        revenue_product
),
taxes AS (
    SELECT
        date,
        SUM(revenue_tax) AS tax
    FROM
        product_taxes
    GROUP BY
        date
),
A AS (
    SELECT
        a.date,
        a.revenue,
        b.expense AS costs,
        c.tax,
        a.revenue - b.expense - c.tax AS gross_profit
    FROM
        revenue_day AS a
        LEFT JOIN expenses AS b
        ON a.date = b.date
        LEFT JOIN taxes AS c
        ON a.date = c.date
), 
B AS (
    SELECT
        date,
        revenue,
        costs,
        tax,
        gross_profit,
        SUM(revenue) OVER (ORDER BY date) AS total_revenue,
        SUM(costs) OVER (ORDER BY date) AS total_costs,
        SUM(tax) OVER (ORDER BY date) AS total_tax,
        SUM(gross_profit) OVER (ORDER BY date) AS total_gross_profit
    FROM
        A
) 
SELECT
    date,
    revenue,
    costs,
    tax,
    gross_profit,
    total_revenue,
    total_costs,
    total_tax,
    total_gross_profit,
    ROUND((gross_profit::decimal * 100 ) / revenue, 2) AS gross_profit_ratio,
    ROUND((total_gross_profit::decimal * 100) / total_revenue, 2) AS total_gross_profit_ratio
FROM 
    B
ORDER BY
    date

In [7]:
file_path7 = ('data/6.csv')
df7 = pd.read_csv(file_path7)
print(df7)

        date  revenue    costs        tax  gross_profit  total_revenue  \
0   24/08/22    49924   159120    6334.09    -115530.09          49924   
1   25/08/22   430860   447560   53545.01     -70245.01         480784   
2   26/08/22   534766   565680   66229.97     -97143.97        1015550   
3   27/08/22   817053   781040  102245.77     -66232.77        1832603   
4   28/08/22  1133370  1055870  140881.28     -63381.28        2965973   
5   29/08/22  1279891  1144280  160378.17     -24767.17        4245864   
6   30/08/22  1279377  1169140  159020.61     -48783.61        5525241   
7   31/08/22  1312720  1159250  162871.96      -9401.96        6837961   
8   01/09/22  1406101  1180320  175026.05      50754.95        8244062   
9   02/09/22  1907107  1590965  237729.71      78412.29       10151169   
10  03/09/22  2210988  1813235  276335.45     121417.55       12362157   
11  04/09/22  2294009  1885300  287736.54     120972.46       14656166   
12  05/09/22  1784690  1456825  222821

Динамика изменения показателей. 
![Image](./charts/22.png)

Динамика изменения показателей. 
![Image](./charts/23.png)

Начиная с 1 сентября ежедневная валовая прибыль сервиса стала положительной. Нулевую отметку суммарная валовая прибыль сервиса превышает 6 сентября.  
Оптимизация стоимости сборки заказа в сентябре позволила увидеть в этом месяце положительную валовую прибыль, так как начиная именно с сентября валовая прибыль становится положительной. 